# Speciation Solution SymPy Code Generation
### Model specification using repeated expression generation (faster)

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import sympy as sym
import scipy.optimize as opt
from os import path
import sys
import fileinput
from thermoengine import coder, core, phases, model, equilibrate
sym.init_printing()

Redirect all C-level std io and std err output to the notebook.  The command must be issued *prior* to any such output being generated by the kernel

In [ ]:
%load_ext wurlitzer

## Speciation Solution Properties
Assumptions:
- There are $nc$ components in the system
- There are $nc+ns$ species in the system.
 - The species are divided into $nb$ basis species ($nb$ == $nc$) and $ns$ non-basis (dependent) species
- The configurational entropy is described as a simple $x_i log(x_i)$ sum over all species
- The example below assumes ideal mixing amongst species

## Number of solution components, basis species and non-basis species

In [ ]:
nc = 2
nb = nc
ns = 3

## Set test conditions and model type

In [ ]:
t = 1300.0  # K
p =    1.0  # bars
module_type = 'calib' # 'fast'

# Generate example endmembers from coder StdStateModel class

In [ ]:
modelCD = coder.StdStateModel()

In [ ]:
GTP = sym.symbols('GTP')
params = [('GTP','J',GTP)]
modelCD.add_expression_to_model(GTP, params)

In [ ]:
modelCD.set_module_name('gas_species')

In [ ]:
model_working_dir = "working"
!mkdir -p {model_working_dir}
%cd {model_working_dir}

### Basis

In [ ]:
param_dict = {'Phase':'H','Formula':'H(1)','T_r':298.15,'P_r':1.0,'GTP':-100000.0}
result = modelCD.create_code_module(phase=param_dict.pop('Phase', None),
                                    formula=param_dict.pop('Formula', None),
                                    params=param_dict,
                                    module_type=module_type,
                                    silent=True)
%cp gas_species.pyx endmembers.pyx
file_name = '"H_gas_species_calib.c"' if module_type == 'calib' else '"H_gas_species_calc.c"'

In [ ]:
param_dict = {'Phase':'Al','Formula':'Al(1)','T_r':298.15,'P_r':1.0,'GTP':-200000.0}
result = modelCD.create_code_module(phase=param_dict.pop('Phase', None),
                                    formula=param_dict.pop('Formula', None),
                                    params=param_dict,
                                    module_type=module_type,
                                    silent=True)
%cat gas_species.pyx >> endmembers.pyx
file_name += ', "Al_gas_species_calib.c"' if module_type == 'calib' else ', "Al_gas_species_calc.c"'

### Non-basis

In [ ]:
param_dict = {'Phase':'AlH','Formula':'H(1)Al(1)','T_r':298.15,'P_r':1.0,'GTP':-160000.0}
result = modelCD.create_code_module(phase=param_dict.pop('Phase', None),
                                    formula=param_dict.pop('Formula', None),
                                    params=param_dict,
                                    module_type=module_type,
                                    silent=True)
%cat gas_species.pyx >> endmembers.pyx
file_name += ', "AlH_gas_species_calib.c"' if module_type == 'calib' else ', "AlH_gas_species_calc.c"'

In [ ]:
param_dict = {'Phase':'Al2H','Formula':'H(1)Al(2)','T_r':298.15,'P_r':1.0,'GTP':-400000.0}
result = modelCD.create_code_module(phase=param_dict.pop('Phase', None),
                                    formula=param_dict.pop('Formula', None),
                                    params=param_dict,
                                    module_type=module_type,
                                    silent=True)
%cat gas_species.pyx >> endmembers.pyx
file_name += ', "Al2H_gas_species_calib.c"' if module_type == 'calib' else ', "Al2H_gas_species_calc.c"'

In [ ]:
param_dict = {'Phase':'AlH2','Formula':'H(2)Al(1)','T_r':298.15,'P_r':1.0,'GTP':-400000.0}
result = modelCD.create_code_module(phase=param_dict.pop('Phase', None),
                                    formula=param_dict.pop('Formula', None),
                                    params=param_dict,
                                    module_type=module_type,
                                    silent=True)
%cat gas_species.pyx >> endmembers.pyx
file_name += ', "AlH2_gas_species_calib.c"' if module_type == 'calib' else ', "AlH2_gas_species_calc.c"'

### Build the endmembers

In [ ]:
%cp endmembers.pyx gas_species.pyx
with open('gas_species.pyxbld', 'r') as f:
    fold = f.read()
    f.close()
if module_type == 'calib':
    fnew = fold.replace("'AlH2_gas_species_calib.c'", file_name)
else:
    fnew = fold.replace("'AlH2_gas_species_calc.c'", file_name)
with open('gas_species.pyxbld', 'w') as f:
    f.write(fnew)
    f.close()

In [ ]:
import gas_species
%cd ..

### Load and test the endmembers

In [ ]:
if module_type == 'calib':
    modelDB = model.Database(database="CoderModule", calib=True, phase_tuple=('gas_species', {
        'H':['H','pure'],
        'Al':['Al','pure'],
        'AlH':['AlH','pure'],
        'Al2H':['Al2H','pure'],
        'AlH2':['AlH2', 'pure']
    }))
else:
    modelDB = model.Database(database="CoderModule", calib=False, phase_tuple=('gas_species', {
        'H':['H','pure'],
        'Al':['Al','pure'],
        'AlH':['AlH','pure'],
        'Al2H':['Al2H','pure'],
        'AlH2':['AlH2', 'pure']
    }))

In [ ]:
H = modelDB.get_phase('H')
Al = modelDB.get_phase('Al')
AlH = modelDB.get_phase('AlH')
Al2H = modelDB.get_phase('Al2H')
AlH2 = modelDB.get_phase('AlH2')
mu0 = np.array([
    H.gibbs_energy(t,p), 
    Al.gibbs_energy(t,p), 
    AlH.gibbs_energy(t,p), 
    Al2H.gibbs_energy(t,p), 
    AlH2.gibbs_energy(t,p)
])
mu0_b = mu0[0:nc]
mu0_s = mu0[nc:]
mu0_b, mu0_s

## Create conversion matrces to map basis to non-basis species
In this case use elements as a component basis (nc).  This does not have to be the case.  It is just convenient to do so here.

In [ ]:
C = np.array([
    H.props['element_comp'][0],
    Al.props['element_comp'][0],
    AlH.props['element_comp'][0],
    Al2H.props['element_comp'][0],
    AlH2.props['element_comp'][0]
])
elm_sys_ind = np.where(np.sum(C,axis=0) > 0)[0]
elm_sys = [core.chem.PERIODIC_ORDER[i] for i in elm_sys_ind]
C = C[:,elm_sys_ind]
print ('Elements in the system:', elm_sys)
print ('C matrix - maps species to elements:')
C

#### Create basis and non-basis sub-matrices of C in SymPy

Partician this matrix into *basis* and *non-basis* submatrices  

Produce a SymPy Matrix symbol (sym.Matrix) from a list of lists (.tolist()) and turn all floating point elements of the matrix into rational numbers (sym.nsimplify(..., rational=True))  

In [ ]:
Cb = sym.nsimplify(sym.Matrix(C[:2,:].tolist()), rational=True)
Cs = sym.nsimplify(sym.Matrix(C[2:,:].tolist()), rational=True)
Cb, Cs

Form a reaction matrix, mapping basis species (columns) to non-basis species (rows)

In [ ]:
R = Cs * (Cb**-1)
R

# Create a speciation solution model

In [ ]:
model = coder.SpeciationSolnModel(nc=nc, ns=ns, Cb=sym.matrix2numpy(Cb), Cs=sym.matrix2numpy(Cs), R=R, debug_print=True)

## Retrieve primary compositional variables
- $n$ is a vector of mole numbers of each component  
- $n_T$ is the total number of moles in the solution
### and construct a derived mole fraction variable
- $X$ is a vector of mole fractions of components in the system

In [ ]:
n = model.n
nT = model.nT
X = n/nT
n, nT, X

## Retrieve the temperature, pressure, and standard state chemical potentials
- $T$ is temperature in $K$
- $P$ is pressure in $bars$
- $\mu$ in Joules

In [ ]:
T = model.get_symbol_for_t()
P = model.get_symbol_for_p()
mu = model.mu
mu_ex = model.mu_ex
T,P,mu,mu_ex

*Rg* is the gas constant

In [ ]:
Rg = sym.symbols('Rg')
params = [['Rg', 'J/K', Rg]]

## Gibbs free energy contribution for the basis species

In [ ]:
G_basis = (n.transpose()*mu[:nb,:])[0] + (n.transpose()*mu_ex[:nb,:])[0]
for i in range(0,nc):
    G_basis += Rg*T*nT*X[i]*sym.log(X[i])
G_basis

Add this expression to the model

In [ ]:
model.add_expression_to_model(G_basis, params)

## Gibbs free energy contribution for the non-basis species 
Formulate this expression as a loop over all basis species  
First, retrieve:
- An index variable for the loop
- A matrix of index-based symbols that correspond to rows of the R matrix
- A placeholder function for the standard state chemical potential of a non-basis species

In [ ]:
r = model.r
mu_s = model.mu_s
mu_e = model.mu_e
j = model.j

In [ ]:
r, mu_s, mu_e, j

In [ ]:
r1 = r[0]
r2 = r[1]

Using the law of mass action, construct an expression that relates the chemical potential of the basis and non-basis species

In [ ]:
lnQ = - (mu_s + mu_e - r1[j]*(mu[0]+mu_ex[0]) - r2[j]*(mu[1]+mu_ex[1]))/(Rg*T)
Q = sym.exp(lnQ)
lnQ, Q

On the assumption of ideal mixing, express the mole fraction of each non-basis species in terms of mole fractions of the basis species and a row, $r$, and the reaction coefficient matrix, $R$.

In [ ]:
Xs = Q*(X[0]**r1[j])*(X[1]**r2[j])
RTlnXs = Rg*T*lnQ + Rg*T*(r1[j]*sym.log(X[0])+r2[j]*sym.log(X[1]))
Xs, RTlnXs

The following term gives the ideal contribution to the Gibbs free energy of solution for a non-basis species

In [ ]:
G_non_basis_term = nT*Xs*(mu_s + RTlnXs)
G_non_basis_term

In [ ]:
model.add_expression_to_model(G_non_basis_term, [])

## Set up the model ...

In [ ]:
model.module = "Speciated_Solution"

... assign a formula string for code generation  

In [ ]:
formula_string = ''
conversion_string_l = []
test_string_l = []
for i, x in enumerate(elm_sys):
    formula_string += x + '[' + x + ']'
    conversion_string_l.append('[' + str(i) + ']=[' + x + ']')
    test_string_l.append('[' + str(i) + '] > 0.0')
formula_string, conversion_string_l, test_string_l

... assign a conversion string to map element concentrations to moles of end members

In [ ]:
model.formula_string = formula_string
model.conversion_string = conversion_string_l
model.test_string = test_string_l

# Use code printers to construct "C" package code

In [ ]:
model_working_dir = "working"
!mkdir -p {model_working_dir}
%cd {model_working_dir}

## Choose model type and create model
model_type is "fast" or "calib"

In [ ]:
model_type = "calib"
paramValues = {"Rg":8.3143, "T_r":298.15, "P_r":1.0}
file_name.replace('"', '').replace('_calib.c', '').replace(' ', '').split(',')

In [ ]:
model.create_code_module(phase="IdealGas", params=paramValues, 
                         endmembers=file_name.replace('"', '').replace('_calib.c', '').replace(' ', '').split(','), 
                         prefix="cy", module_type=model_type, silent=False, minimal_deriv_set=True, debug=1)

## Load the module

In [ ]:
import Speciated_Solution
%cd ..

## Test and time the generated functions for IdealGas (T in K, P in bars)

In [ ]:
t = 1300.00
p = 1.0
n = np.array([2.0, 2.0])

## Available in both "Fast" and "Calib" code versions 
Execute the "fast" or "calibration" code metadata retrieval functions:

In [ ]:
try:
    print(Speciated_Solution.cy_IdealGas_Speciated_Solution_identifier())
    print(Speciated_Solution.cy_IdealGas_Speciated_Solution_name())
    print(Speciated_Solution.cy_IdealGas_Speciated_Solution_formula(t,p,n))
except AttributeError:
    pass
try:
    print(Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_identifier())
    print(Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_name())
    print(Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_formula(t,p,n))
except AttributeError:
    pass

Test intrinsic element conversion routine ...

In [ ]:
try:
    e = np.zeros(106)
    sum = np.sum(n)
    for index in range(0,nc):
        end = Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_elements(index)
        for i in range(0,106):
            e[i] += end[i]*n[index]/sum
    nConv = Speciated_Solution.cy_IdealGas_Speciated_Solution_conv_elm_to_moles(e)
    for i in range(0,nc):
        print ('X[{0:d}] input {1:13.6e}, calc {2:13.6e}, diff {3:13.6e}'.format(
        i, n[i]/sum, nConv[i], nConv[i]-n[i]/sum))
    if not Speciated_Solution.cy_IdealGas_Speciated_Solution_test_moles(nConv):
        print ('Output of intrinsic composition calculation fails tests for permissible values.')
except AttributeError:
    pass
try:
    e = np.zeros(106)
    sum = np.sum(n)
    for index in range(0,nc):
        end = Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_elements(index)
        for i in range(0,106):
            e[i] += end[i]*n[index]/sum
    nConv = Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_conv_elm_to_moles(e)
    for i in range(0,nc):
        print ('X[{0:d}] input {1:13.6e}, calc {2:13.6e}, diff {3:13.6e}'.format(
        i, n[i]/sum, nConv[i], nConv[i]-n[i]/sum))
    if not Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_test_moles(nConv):
        print ('Output of intrinsic composition calculation fails tests for permissible values.')
except AttributeError:
    pass

Test various conversion routines ...

In [ ]:
try:
    print (Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_conv_moles_to_tot_moles(n))
    print (Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_conv_moles_to_mole_frac(n))
    e = Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_conv_moles_to_elm(n)
    print (e)
    print (Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_conv_elm_to_moles(e))
    print (Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_conv_elm_to_tot_moles(e))
    print (Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_conv_elm_to_tot_grams(e))
except AttributeError:
    pass
try:
    print (Speciated_Solution.cy_IdealGas_Speciated_Solution_conv_moles_to_tot_moles(n))
    print (Speciated_Solution.cy_IdealGas_Speciated_Solution_conv_moles_to_mole_frac(n))
    e = Speciated_Solution.cy_IdealGas_Speciated_Solution_conv_moles_to_elm(n)
    print (e)
    print (Speciated_Solution.cy_IdealGas_Speciated_Solution_conv_elm_to_moles(e))
    print (Speciated_Solution.cy_IdealGas_Speciated_Solution_conv_elm_to_tot_moles(e))
    print (Speciated_Solution.cy_IdealGas_Speciated_Solution_conv_elm_to_tot_grams(e))
except AttributeError:
    pass

### Execute the standard thermodynamic property retrieval functions:

In [ ]:
Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_g(t,p,n)

In [ ]:
fmt = "{0:<10.10s} {1:13.6e} {2:<10.10s}"
try:
    print(fmt.format('G', Speciated_Solution.cy_IdealGas_Speciated_Solution_g(t,p,n), 'J'))
    print(fmt.format('dGdT', Speciated_Solution.cy_IdealGas_Speciated_Solution_dgdt(t,p,n), 'J/K'))
    print(fmt.format('dGdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_dgdp(t,p,n), 'J/bar'))
    print(fmt.format('d2GdT2', Speciated_Solution.cy_IdealGas_Speciated_Solution_d2gdt2(t,p,n), 'J/K^2'))
    print(fmt.format('d2GdTdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_d2gdtdp(t,p,n), 'J/K-bar'))
    print(fmt.format('d2GdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_d2gdp2(t,p,n), 'J/bar^2'))
    print(fmt.format('d3GdT3', Speciated_Solution.cy_IdealGas_Speciated_Solution_d3gdt3(t,p,n), 'J/K^3'))
    print(fmt.format('d3GdT2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_d3gdt2dp(t,p,n), 'J/K^2-bar'))
    print(fmt.format('d3GdTdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_d3gdtdp2(t,p,n), 'J/K-bar^2'))
    print(fmt.format('d3GdP3', Speciated_Solution.cy_IdealGas_Speciated_Solution_d3gdp3(t,p,n), 'J/bar^3'))
    print(fmt.format('S', Speciated_Solution.cy_IdealGas_Speciated_Solution_s(t,p,n), 'J/K'))
    print(fmt.format('V', Speciated_Solution.cy_IdealGas_Speciated_Solution_v(t,p,n), 'J/bar'))
    print(fmt.format('Cv', Speciated_Solution.cy_IdealGas_Speciated_Solution_cv(t,p,n), 'J/K'))
    print(fmt.format('Cp', Speciated_Solution.cy_IdealGas_Speciated_Solution_cp(t,p,n), 'J/K'))
    print(fmt.format('dCpdT', Speciated_Solution.cy_IdealGas_Speciated_Solution_dcpdt(t,p,n), 'J/K^2'))
    print(fmt.format('alpha', Speciated_Solution.cy_IdealGas_Speciated_Solution_alpha(t,p,n), '1/K'))
    print(fmt.format('beta', Speciated_Solution.cy_IdealGas_Speciated_Solution_beta(t,p,n), '1/bar'))
    print(fmt.format('K', Speciated_Solution.cy_IdealGas_Speciated_Solution_K(t,p,n), 'bar'))
    print(fmt.format('Kp', Speciated_Solution.cy_IdealGas_Speciated_Solution_Kp(t,p,n), ''))
except AttributeError:
    pass
try:
    print(fmt.format('G', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_g(t,p,n), 'J'))
    print(fmt.format('dGdT', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_dgdt(t,p,n), 'J/K'))
    print(fmt.format('dGdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_dgdp(t,p,n), 'J/bar'))
    print(fmt.format('d2GdT2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d2gdt2(t,p,n), 'J/K^2'))
    print(fmt.format('d2GdTdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d2gdtdp(t,p,n), 'J/K-bar'))
    print(fmt.format('d2GdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d2gdp2(t,p,n), 'J/bar^2'))
    print(fmt.format('d3GdT3', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d3gdt3(t,p,n), 'J/K^3'))
    print(fmt.format('d3GdT2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d3gdt2dp(t,p,n), 'J/K^2-bar'))
    print(fmt.format('d3GdTdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d3gdtdp2(t,p,n), 'J/K-bar^2'))
    print(fmt.format('d3GdP3', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d3gdp3(t,p,n), 'J/bar^3'))
    print(fmt.format('S', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_s(t,p,n), 'J/K'))
    print(fmt.format('V', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_v(t,p,n), 'J/bar'))
    print(fmt.format('Cv', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_cv(t,p,n), 'J/K'))
    print(fmt.format('Cp', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_cp(t,p,n), 'J/K'))
    print(fmt.format('dCpdT', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_dcpdt(t,p,n), 'J/K^2'))
    print(fmt.format('alpha', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_alpha(t,p,n), '1/K'))
    print(fmt.format('beta', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_beta(t,p,n), '1/bar'))
    print(fmt.format('K', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_K(t,p,n), 'bar'))
    print(fmt.format('Kp', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_Kp(t,p,n), ''))
except AttributeError:
    pass

### Execute functions that access endmember properties:

In [ ]:
fmt = "{0:<10.10s} {1:13.6e} {2:<15.15s}"
try:
    print ("number of components", Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_number())
    for index in range(0, nc):
        print ("{0:<20.20s}".format(Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_name(index)), end=' ')
        print ("{0:<20.20s}".format(Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_formula(index)))
        print ("mw: {0:10.2f}".format(Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_mw(index)))
        print (fmt.format('mu0', Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_mu0(index,t,p), 'J/mol'))
        print (fmt.format('dmu0dT', Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_dmu0dT(index,t,p), 'J/K-mol'))
        print (fmt.format('dmu0dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_dmu0dP(index,t,p), 'J/bar-mol'))
        print (fmt.format('d2mu0dT2', Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_d2mu0dT2(index,t,p), 'J/K^2-mol'))
        print (fmt.format('d2mu0dTdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_d2mu0dTdP(index,t,p), 'J/K-bar-mol'))
        print (fmt.format('d2mu0dP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_d2mu0dP2(index,t,p), 'J/bar^2-mol'))
        print (fmt.format('d3mu0dT3', Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_d3mu0dT3(index,t,p), 'J/K^3-mol'))
        print (fmt.format('d3mu0dT2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_d3mu0dT2dP(index,t,p), 'J/K^2-bar-mol'))
        print (fmt.format('d3mu0dTdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_d3mu0dTdP2(index,t,p), 'J/K-bar^2-mol'))
        print (fmt.format('d3mu0dP3', Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_d3mu0dP3(index,t,p), 'J/bar^3-mol'))
        print ("Element array:")
        print (Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_elements(index))
        print ()
except AttributeError:
    pass
try:
    print ("number of components", Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_number())
    for index in range(0, nc):
        print ("{0:<20.20s}".format(Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_name(index)), end=' ')
        print ("{0:<20.20s}".format(Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_formula(index)), end=' ')
        print ("mw: {0:10.2f}".format(Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_mw(index)))
        print (fmt.format('mu0', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_mu0(index,t,p), 'J/mol'))
        print (fmt.format('dmu0dT', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_dmu0dT(index,t,p), 'J/K-mol'))
        print (fmt.format('dmu0dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_dmu0dP(index,t,p), 'J/bar-mol'))
        print (fmt.format('d2mu0dT2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_d2mu0dT2(index,t,p), 'J/K^2-mol'))
        print (fmt.format('d2mu0dTdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_d2mu0dTdP(index,t,p), 'J/K-bar-mol'))
        print (fmt.format('d2mu0dP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_d2mu0dP2(index,t,p), 'J/bar^2-mol'))
        print (fmt.format('d3mu0dT3', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_d3mu0dT3(index,t,p), 'J/K^3-mol'))
        print (fmt.format('d3mu0dT2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_d3mu0dT2dP(index,t,p), 'J/K^2-bar-mol'))
        print (fmt.format('d3mu0dTdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_d3mu0dTdP2(index,t,p), 'J/K-bar^2-mol'))
        print (fmt.format('d3mu0dP3', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_d3mu0dP3(index,t,p), 'J/bar^3-mol'))
        print ("Element array:")
        print (Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_elements(index))
        print ()
except AttributeError:
    pass

### Execute functions that access species properties:

In [ ]:
fmt = "{0:<10.10s} {1:13.6e} {2:<15.15s}"
try:
    print ("number of species", Speciated_Solution.cy_IdealGas_Speciated_Solution_species_number())
    for index in range(0, nc):
        print ("{0:<20.20s}".format(Speciated_Solution.cy_IdealGas_Speciated_Solution_species_name(index)), end=' ')
        print ("{0:<20.20s}".format(Speciated_Solution.cy_IdealGas_Speciated_Solution_species_formula(index)))
        print ("mw: {0:10.2f}".format(Speciated_Solution.cy_IdealGas_Speciated_Solution_species_mw(index)))
        print ("Element array:")
        print (Speciated_Solution.cy_IdealGas_Speciated_Solution_species_elements(index))
        print ()
except AttributeError:
    pass
try:
    print ("number of species", Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_species_number())
    for index in range(0, nc):
        print ("{0:<20.20s}".format(Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_species_name(index)), end=' ')
        print ("{0:<20.20s}".format(Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_species_formula(index)), end=' ')
        print ("mw: {0:10.2f}".format(Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_species_mw(index)))
        print ("Element array:")
        print (Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_species_elements(index))
        print ()
except AttributeError:
    pass

### Execute functions for molar derivatives
#### First derivative vectors:

In [ ]:
def printResult(name, result, units):
    print ("{0:<10.10s}".format(name), end=' ')
    [print ("{0:13.6e}".format(x), end=' ') for x in result]
    print ("{0:<10.10s}".format(units))
def printLabels(n):
    print ("{0:<18.18s}".format(''), end=' ')
    [print ("[{0:3d}]{1:<8.8s}".format(idx, ''), end=' ') for idx in range(len(n))]
    print ()
printLabels(n)
try:
    printResult('dGdn', Speciated_Solution.cy_IdealGas_Speciated_Solution_dgdn(t,p,n), 'J/m')
    printResult('d2GdndT', Speciated_Solution.cy_IdealGas_Speciated_Solution_d2gdndt(t,p,n), 'J/K-m')
    printResult('d2GdndP', Speciated_Solution.cy_IdealGas_Speciated_Solution_d2gdndp(t,p,n), 'J/bar-m')
    printResult('d3GdndT2', Speciated_Solution.cy_IdealGas_Speciated_Solution_d3gdndt2(t,p,n), 'J/K^2-m')
    printResult('d3GdndTdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_d3gdndtdp(t,p,n), 'J/K-bar-m')
    printResult('d3GdndP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_d3gdndp2(t,p,n), 'J/bar^2-m')
    printResult('d4GdndT3', Speciated_Solution.cy_IdealGas_Speciated_Solution_d4gdndt3(t,p,n), 'J/K^3-m')
    printResult('d4GdndT2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_d4gdndt2dp(t,p,n), 'J/K^2-bar-m')
    printResult('d4GdndTdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_d4gdndtdp2(t,p,n), 'J/K-bar^2-m')
    printResult('d4GdndP3', Speciated_Solution.cy_IdealGas_Speciated_Solution_d4gdndp3(t,p,n), 'J/bar^3-m')
except AttributeError:
    pass
try:
    printResult('dGdn', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_dgdn(t,p,n), 'J/m')
    printResult('d2GdndT', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d2gdndt(t,p,n), 'J/K-m')
    printResult('d2GdndP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d2gdndp(t,p,n), 'J/bar-m')
    printResult('d3GdndT2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d3gdndt2(t,p,n), 'J/K^2-m')
    printResult('d3GdndTdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d3gdndtdp(t,p,n), 'J/K-bar-m')
    printResult('d3GdndP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d3gdndp2(t,p,n), 'J/bar^2-m')
    printResult('d4GdndT3', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d4gdndt3(t,p,n), 'J/K^3-m')
    printResult('d4GdndT2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d4gdndt2dp(t,p,n), 'J/K^2-bar-m')
    printResult('d4GdndTdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d4gdndtdp2(t,p,n), 'J/K-bar^2-m')
    printResult('d4GdndP3', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d4gdndp3(t,p,n), 'J/bar^3-m')
except AttributeError:
    pass    

#### The Hessian matrix (molar second derivative matrix) is stored as a compact linear array
A function is provided to map matrix indices to compact storage 1-D array indices

In [ ]:
for i in range(1,nc+1):
    print ("[ ", end=' ')
    for j in range (1,nc+1):
        print ((i,j), end=' ')
    print (']     [', end=' ')
    for j in range (1,nc+1):
        print (model.symmetric_index_from_2d_array(elm=(i,j)), end=' ')
    print (']')

In [ ]:
def printResult(name, result, units):
    print ("{0:<10.10s}".format(name), end=' ')
    [print ("{0:13.6e}".format(x), end=' ') for x in result]
    print ("{0:<10.10s}".format(units))
def printLabels(n):
    print ("{0:<18.18s}".format(''), end=' ')
    maxIdx = int(len(n)*(len(n)-1)/2 + len(n))
    [print ("[{0:3d}]{1:<8.8s}".format(idx, ''), end=' ') for idx in range(maxIdx)]
    print ()
printLabels(n)
try:
    printResult('d2Gdn2', Speciated_Solution.cy_IdealGas_Speciated_Solution_d2gdn2(t,p,n), 'J/m^2')
    printResult('d3Gdn2dT', Speciated_Solution.cy_IdealGas_Speciated_Solution_d3gdn2dt(t,p,n), 'J/K-m^2')
    printResult('d3Gdn2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_d3gdn2dp(t,p,n), 'J/bar-m^2')
    printResult('d4Gdn2dT2', Speciated_Solution.cy_IdealGas_Speciated_Solution_d4gdn2dt2(t,p,n), 'J/K^2-m^2')
    printResult('d4Gdn2dTdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_d4gdn2dtdp(t,p,n), 'J/K-bar-m^2')
    printResult('d4Gdn2dP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_d4gdn2dp2(t,p,n), 'J/bar^2-m^2')
    printResult('d5Gdn2dT3', Speciated_Solution.cy_IdealGas_Speciated_Solution_d5gdn2dt3(t,p,n), 'J/K^3-m^2')
    printResult('d5Gdn2dT2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_d5gdn2dt2dp(t,p,n), 'J/K^2-bar-m^2')
    printResult('d5Gdn2dTdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_d5gdn2dtdp2(t,p,n), 'J/K-bar^2-m^2')
    printResult('d5Gdn2dP3', Speciated_Solution.cy_IdealGas_Speciated_Solution_d5gdn2dp3(t,p,n), 'J/bar^3-m^2')
except AttributeError:
    pass
try:
    printResult('d2Gdn2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d2gdn2(t,p,n), 'J/m^2')
    printResult('d3Gdn2dT', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d3gdn2dt(t,p,n), 'J/K-m^2')
    printResult('d3Gdn2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d3gdn2dp(t,p,n), 'J/bar-m^2')
    printResult('d4Gdn2dT2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d4gdn2dt2(t,p,n), 'J/K^2-m^2')
    printResult('d4Gdn2dTdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d4gdn2dtdp(t,p,n), 'J/K-bar-m^2')
    printResult('d4Gdn2dP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d4gdn2dp2(t,p,n), 'J/bar^2-m^2')
    printResult('d5Gdn2dT3', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d5gdn2dt3(t,p,n), 'J/K^3-m^2')
    printResult('d5Gdn2dT2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d5gdn2dt2dp(t,p,n), 'J/K^2-bar-m^2')
    printResult('d5Gdn2dTdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d5gdn2dtdp2(t,p,n), 'J/K-bar^2-m^2')
    printResult('d5Gdn2dP3', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d5gdn2dp3(t,p,n), 'J/bar^3-m^2')
except AttributeError:
    pass

#### The 3-D Tensor (molar third derivative tensor) is stored as a compact linear array
A function is provided to map matrix indices to compact storage 1-D array indices:  
If $n_c$ represents the number of components in the solution, and  
if $n_d$ represents the dimensionality of molar derivative (in this case 3), then  
the number of numerically ordered permutations of $n_c$ molar derivatives taken $n_d$ at a time is:

In [ ]:
n_c,n_d = sym.symbols('n_c n_d')
q = sym.factorial(n_c+n_d-1)/sym.factorial(n_d)/sym.factorial(n_c-1)
q

Substituting $n_d$ equal to 3 and simplifying gives:

In [ ]:
q = sym.simplify(q.subs(n_d,3))
q

and, for the number of components in this solution, there will be the following number of unique terms in the third derivative tensor:

In [ ]:
q.subs(n_c,nc)

A function is provided to map matrix indices to compact storage 1-D array indices

In [ ]:
for i in range(1,nc+1):
    for j in range (1,nc+1):
        print ("[", end=' ')
        for k in range (1,nc+1):
            print ("{0:1d}{1:1d}{2:1d}".format(i,j,k), end=' ')
        print ('] ', end=' ')
    print ('  ->  ', end=' ')
    for j in range (1,nc+1):
        print ("[", end=' ')
        for k in range (1,nc+1):
            print (model.symmetric_index_from_3d_array(elm=(i,j,k)), end=' ')
        print ('] ', end=' ')
    print ('')

In [ ]:
def printResult(name, result, units):
    print ("{0:<10.10s}".format(name), end=' ')
    [print ("{0:10.3e}".format(x), end=' ') for x in result]
    print ("{0:<14.14s}".format(units))
def printLabels(n):
    print ("{0:<15.15s}".format(''), end=' ')
    maxIdx = int(len(n)*(len(n)+1)*(len(n)+2)/6)
    [print ("[{0:3d}]{1:<5.5s}".format(idx, ''), end=' ') for idx in range(maxIdx)]
    print ()
printLabels(n)
try:
    printResult('d3Gdn3', Speciated_Solution.cy_IdealGas_Speciated_Solution_d3gdn3(t,p,n), 'J/m^3')
    printResult('d4Gdn3dT', Speciated_Solution.cy_IdealGas_Speciated_Solution_d4gdn3dt(t,p,n), 'J/K-m^3')
    printResult('d4Gdn3dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_d4gdn3dp(t,p,n), 'J/bar-m^3')
    printResult('d5Gdn3dT2', Speciated_Solution.cy_IdealGas_Speciated_Solution_d5gdn3dt2(t,p,n), 'J/K^2-m^3')
    printResult('d5Gdn3dTdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_d5gdn3dtdp(t,p,n), 'J/K-bar-m^3')
    printResult('d5Gdn3dP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_d5gdn3dp2(t,p,n), 'J/bar^2-m^3')
    printResult('d6Gdn3dT3', Speciated_Solution.cy_IdealGas_Speciated_Solution_d6gdn3dt3(t,p,n), 'J/K^3-m^3')
    printResult('d6Gdn3dT2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_d6gdn3dt2dp(t,p,n), 'J/K^2-bar-m^3')
    printResult('d6Gdn3dTdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_d6gdn3dtdp2(t,p,n), 'J/K-bar^2-m^3')
    printResult('d6Gdn3dP3', Speciated_Solution.cy_IdealGas_Speciated_Solution_d6gdn3dp3(t,p,n), 'J/bar^3-m^3')
except AttributeError:
    pass
try:
    printResult('d3Gdn3', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d3gdn3(t,p,n), 'J/m^3')
    printResult('d4Gdn3dT', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d4gdn3dt(t,p,n), 'J/K-m^3')
    printResult('d4Gdn3dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d4gdn3dp(t,p,n), 'J/bar-m^3')
    printResult('d5Gdn3dT2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d5gdn3dt2(t,p,n), 'J/K^2-m^3')
    printResult('d5Gdn3dTdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d5gdn3dtdp(t,p,n), 'J/K-bar-m^3')
    printResult('d5Gdn3dP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d5gdn3dp2(t,p,n), 'J/bar^2-m^3')
    printResult('d6Gdn3dT3', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d6gdn3dt3(t,p,n), 'J/K^3-m^3')
    printResult('d6Gdn3dT2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d6gdn3dt2dp(t,p,n), 'J/K^2-bar-m^3')
    printResult('d6Gdn3dTdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d6gdn3dtdp2(t,p,n), 'J/K-bar^2-m^3')
    printResult('d6Gdn3dP3', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d6gdn3dp3(t,p,n), 'J/bar^3-m^3')
except AttributeError:
    pass

## Test and time the generated functions for IdealGas

Time the code

In [ ]:
try:
    %timeit Speciated_Solution.cy_IdealGas_Speciated_Solution_g(t, p, n)
except AttributeError:
    pass
try:
    %timeit Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_g(t, p, n) 
except AttributeError:
    pass

## Methods available only in the "Calib" versions of generated code
### Execute the parameter value/metadata functions.  
These functions are only defined for the "calibration" model code implementation:

In [ ]:
nparam = 0

In [ ]:
try:
    nparam = Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_number()
    names = Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_names()
    units = Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_units()
    values = Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_values()
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,nparam):
        print(fmt.format(names[i], values[i], Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_value(i), units[i]))
except AttributeError:
    pass

### Functions that allow modification of the array of parameter values

In [ ]:
try:
    values[1] = 100.0
    Speciated_Solution.cy_IdealGas_Speciated_Solution_set_param_values(values)
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,nparam):
        print(fmt.format(names[i], values[i], Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_value(i), units[i]))
except (AttributeError, NameError):
    pass

### Functions that allow modification of a particular parameter value

In [ ]:
try:
    Speciated_Solution.cy_IdealGas_Speciated_Solution_set_param_value(1, 1.0)
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,nparam):
        print(fmt.format(names[i], values[i], Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_value(i), units[i]))
except AttributeError:
    pass

### Functions that evaluate parameter derivatives ...

In [ ]:
try:
    fmt = "    {0:<10.10s} {1:13.6e}"
    for i in range(0, nparam):
        print ('Derivative with respect to parameter: ', names[i], ' of')
        print (fmt.format('G', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_g(t, p, n, i)))
        print (fmt.format('dGdT', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_dgdt(t, p, n, i)))
        print (fmt.format('dGdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_dgdp(t, p, n, i)))
        print (fmt.format('d2GdT2', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d2gdt2(t, p, n, i)))
        print (fmt.format('d2GdTdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d2gdtdp(t, p, n, i)))
        print (fmt.format('d2GdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d2gdp2(t, p, n, i)))
        print (fmt.format('d3GdT3', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d3gdt3(t, p, n, i)))
        print (fmt.format('d3GdT2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d3gdt2dp(t, p, n, i)))
        print (fmt.format('d3GdTdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d3gdtdp2(t, p, n, i)))
        print (fmt.format('d3GdP3', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d3gdp3(t, p, n, i)))
except (AttributeError, TypeError):
    pass

### Parameter derivatives of the chemical potential

In [ ]:
def printResult(name, result, units):
    print ("dmu[*]/d {0:<10.10s}".format(name), end=' ')
    [print ("{0:13.6e}".format(x), end=' ') for x in result]
    print ("{0:<12.12s}".format(units))
def printLabels(n):
    print ("         {0:<18.18s}".format(''), end=' ')
    [print ("[{0:3d}]{1:<8.8s}".format(idx, ''), end=' ') for idx in range(len(n))]
    print ()
try:
    printLabels(n)
    for i in range(0, nparam):
        result = Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_dgdn(t,p,n, i)
        printResult(names[i], result, 'J/m^2/p-unit')
except AttributeError:
    pass    

### Execute the parameter value/metadata functions for endmembers.  
Use the Potassium IdealGas as an example.  Alternatively, other endmembers can be accessed at:
- Speciated_Solution.cy_High_Albite_berman_(method ...)
- Speciated_Solution.cy_Anorthite_berman_(method ...)
- Speciated_Solution.cy_Potassium_IdealGas_berman_(method ...)

In [ ]:
try:
    np = Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_number()
    names = Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_names()
    units = Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_units()
    values = Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_values()
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,np):
        print(fmt.format(names[i], values[i], Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_value(i), units[i]))
except AttributeError:
    pass

Test the functions that allow modification of the array of parameter values

In [ ]:
try:
    values[1] = 100.0
    Speciated_Solution.cy_IdealGas_Speciated_Solution_set_param_values(values)
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,np):
        print(fmt.format(names[i], values[i], Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_value(i), units[i]))
except (AttributeError, NameError):
    pass

Test the functions that allow modification of a particular parameter value

In [ ]:
try:
    Speciated_Solution.cy_IdealGas_Speciated_Solution_set_param_value(1, 1.0)
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,np):
        print(fmt.format(names[i], values[i], Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_value(i), units[i]))
except AttributeError:
    pass

Evaluate parameter derivatives ...

In [ ]:
try:
    fmt = "    {0:<10.10s} {1:13.6e}"
    for i in range(0, np):
        print ('Derivative with respect to parameter: ', names[i], ' of')
        print (fmt.format('G', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_g(t, p, n, i)))
        print (fmt.format('dGdT', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_dgdt(t, p, n, i)))
        print (fmt.format('dGdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_dgdp(t, p, n, i)))
        print (fmt.format('d2GdT2', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d2gdt2(t, p, n, i)))
        print (fmt.format('d2GdTdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d2gdtdp(t, p, n, i)))
        print (fmt.format('d2GdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d2gdp2(t, p, n, i)))
        print (fmt.format('d3GdT3', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d3gdt3(t, p, n, i)))
        print (fmt.format('d3GdT2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d3gdt2dp(t, p, n, i)))
        print (fmt.format('d3GdTdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d3gdtdp2(t, p, n, i)))
        print (fmt.format('d3GdP3', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d3gdp3(t, p, n, i)))
except (AttributeError, TypeError):
    pass

### Alter an endmember thermodynamic parameter value and test to insure that alteration propagates to the solution phase
First output the Gibbs energy of solution with default parameters ...

In [ ]:
try:
    fmt = "{0:<10.10s} {1:13.6e} {2:<10.10s}"
    print(fmt.format('G', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_g(t,p,n), 'J'))
except (AttributeError, TypeError):
    pass

Second, output the reference state enthalpy of formation of the potassium IdealGas end member, then alter it by 10,000 J

In [ ]:
try:
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    names = Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_names()
    units = Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_units()
    values = Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_values()
    print(fmt.format(names[2], values[2], Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_value(2), units[2]))
    Speciated_Solution.cy_IdealGas_Speciated_Solution_set_param_value(2, values[2]+10000.0)
    print(fmt.format(names[2], values[2], Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_value(2), units[2]))
except AttributeError:
    pass

Finally, output the Gibbs energy of solution again to reflect the endmember parameter change

In [ ]:
try:
    fmt = "{0:<10.10s} {1:13.6e} {2:<10.10s}"
    print(fmt.format('G', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_g(t,p,n), 'J'))
except (AttributeError, TypeError):
    pass

# TESTING

In [ ]:
from sympy.printing.ccode import C99CodePrinter, ccode
from sympy.utilities.codegen import codegen

In [ ]:
component_string = ''
endmember_dict = { "Agamma":"Agamma", "Bgamma":"Bgamma",
        "AsubG":"AsubG", "AsubH":"AsubH", "AsubV":"AsubV", "AsubJ":"AsubJ",
        "AsubKappa":"AsubKappa", "AsubEx":"AsubEx",
        "BsubG":"BsubG", "BsubH":"BsubH", "BsubV":"BsubV", "BsubJ":"BsubJ",
        "BsubKappa":"BsubKappa", "BsubEx":"BsubEx" }
ss_list = []
for i in range(1,nc+1):
    component_string += 'n' + str(i) + ' '
    ss_string = 'mu' + str(i)
    ss_list.append(sym.Function(ss_string)(T,P))
    endmember_dict[ss_string] = '(*endmember[' + str(i-1) + '].mu0)'
endmember_dict['mu_s'] = '(*endmember[' + str(nb) + '+j-1].mu0)'

In [ ]:
from thermoengine.coder import SubCodePrinter as ScP
class SubCodePrinter(ScP):
    def __init__(self, settings=None, nBasis=0, forIndex='i'):
        super().__init__(settings=settings)
        self._nBasis = nBasis
        self._forIndex = forIndex
    
    @property
    def nBasis(self):
        return self._nBasis
    @nBasis.setter
    def nBasis(self, nBasis):
        self._nBasis = nBasis
    
    @property
    def forIndex(self):
        return self._forIndex
    @forIndex.setter
    def forIndex(self, forIndex):
        self._forIndex = forIndex
    
    def _print_Sum(self, expr):
        ind = str(expr.limits[0][0])
        low = str(expr.limits[0][1])
        high = str(expr.limits[0][2])
        func = expr.function
        result =  '{\n'
        result += '  double sum = 0.0;\n'
        result += '  for (int '+ind+'='+low+'; '+ind+'<='+high+'; '+ind+'++) {\n'
        result += '    sum += ' + printer.doprint(func) + ';\n'
        result += '  }\n'
        result += '  result += sum;\n'
        result += '}\n'
        return result
    def _print_Derivative(self, expr):
        function, *vars = expr.args
        number_of_derivatives = len(expr.args) - 1
        
        if function.func.__name__[0:4] == 'mu_s':
            if number_of_derivatives == 1:
                derivative_string = repr(vars[0][0])
                derivative_order  = '' if vars[0][1] == 1 else str(vars[0][1])
                result = ('(*endmember['+str(self.nBasis)+'+'+self.forIndex+'-1].d' 
                          + derivative_order + 'mu0d' 
                          + derivative_string + derivative_order + ')(T, P)')
            elif number_of_derivatives == 2:
                derivative_string_2 = repr(vars[0][0])
                derivative_order_2  = '' if vars[0][1] == 1 else str(vars[0][1])
                derivative_string_1 = repr(vars[1][0])
                derivative_order_1  = '' if vars[1][1] == 1 else str(vars[1][1])
                derivative_total    = str(vars[0][1]+vars[1][1])
                result = ('(*endmember['+str(self.nBasis)+'+'+self.forIndex+'-1].d' 
                          + derivative_total + 'mu0d' 
                          + derivative_string_1 + derivative_order_1 +'d' 
                          + derivative_string_2 + derivative_order_2 + ')(T, P)')
            else:
                result = ''

        elif function.func.__name__[0:2] == 'mu':
            function_string_index = (
                int(sym.srepr(function).split("'")[1][2:]) - 1)
            if number_of_derivatives == 1:
                derivative_string = repr(vars[0][0])
                derivative_order  = '' if vars[0][1] == 1 else str(vars[0][1])
                result = ('(*endmember[' + str(function_string_index) + '].d' 
                          + derivative_order + 'mu0d' 
                          + derivative_string + derivative_order + ')(T, P)')
            elif number_of_derivatives == 2:
                derivative_string_2 = repr(vars[0][0])
                derivative_order_2  = '' if vars[0][1] == 1 else str(vars[0][1])
                derivative_string_1 = repr(vars[1][0])
                derivative_order_1  = '' if vars[1][1] == 1 else str(vars[1][1])
                derivative_total    = str(vars[0][1]+vars[1][1])
                result = ('(*endmember[' + str(function_string_index) + '].d' 
                          + derivative_total + 'mu0d' 
                          + derivative_string_1 + derivative_order_1 +'d' 
                          + derivative_string_2 + derivative_order_2 + ')(T, P)')
            else:
                result = ''
        
        elif (len(function.func.__name__) >= 6 and 
            function.func.__name__[1:6] == 'gamma'):
            if number_of_derivatives == 1:
                derivative_string = repr(vars[0][0]).lower()
                derivative_order  = '' if vars[0][1] == 1 else str(vars[0][1])
                result = ('d' + derivative_order + function.func.__name__ + 'd' 
                    + derivative_string + derivative_order + '(T, P)')
            elif number_of_derivatives == 2:
                derivative_string_2 = repr(vars[0][0]).lower()
                derivative_order_2  = '' if vars[0][1] == 1 else str(vars[0][1])
                derivative_string_1 = repr(vars[1][0]).lower()
                derivative_order_1  = '' if vars[1][1] == 1 else str(vars[1][1])
                derivative_total    = str(vars[0][1]+vars[1][1])
                result = ('d' + derivative_total + function.func.__name__ + 'D' 
                    + derivative_string_1 + derivative_order_1 +'D' 
                    + derivative_string_2 + derivative_order_2 + '(T, P)')
            else:
                result = ''

        elif (len(function.func.__name__) >= 8 and 
            function.func.__name__[0:9] == 'gSolvent'):
            if number_of_derivatives == 1:
                derivative_string = repr(vars[0][0]).lower()
                derivative_order  = '' if vars[0][1] == 1 else str(vars[0][1])
                result = ('D' + derivative_order + function.func.__name__ + 'D' 
                    + derivative_string + derivative_order + '(T, P)')
            elif number_of_derivatives == 2:
                derivative_string_2 = repr(vars[0][0]).lower()
                derivative_order_2  = '' if vars[0][1] == 1 else str(vars[0][1])
                derivative_string_1 = repr(vars[1][0]).lower()
                derivative_order_1  = '' if vars[1][1] == 1 else str(vars[1][1])
                derivative_total    = str(vars[0][1]+vars[1][1])
                result = ('D' + derivative_total + function.func.__name__ + 'D' 
                    + derivative_string_1 + derivative_order_1 +'D' 
                    + derivative_string_2 + derivative_order_2 + '(T, P)')
            else:
                result = ''

        else:
            if (not isinstance(type(function), UndefinedFunction) or 
                not all(isinstance(i, Symbol) for i in vars)):
                return super()._print_Derivative(expr)
        return result


In [ ]:
printer = SubCodePrinter(settings={'user_functions':endmember_dict})
printer.nBasis = nb
printer.forIndex = 'j'

In [ ]:

print(printer.doprint(R[0,0]))
print(printer.doprint(R[:,1], assign_to='r2'))

In [ ]:
add_R = False
print (add_R)
add_R |= G_basis.has(sym.IndexedBase)
print (add_R)
add_R |= G_non_basis_term.has(sym.IndexedBase)
print (add_R)

In [ ]:
str((ns,nb))

In [ ]:
print(printer.doprint(G_basis.diff(n[0]), assign_to='result'))

In [ ]:
print(printer.doprint(G_non_basis).replace('result', 'G'))

In [ ]:
print(printer.doprint(G_non_basis.diff(n[0])).replace('result', 'dGdn[0]'))

In [ ]:
print(printer.doprint(G_non_basis.diff(T)).replace('result', 'dGdT'))

In [ ]:
print(printer.doprint(G_non_basis.diff(n[0],2)).replace('result', 'd2Gdn2[0][0]'))

In [ ]:
print(printer.doprint(G_non_basis.diff(n[0],3)).replace('result', 'd3Gdn3[0][0][0]'))